<a href="https://colab.research.google.com/github/DarianSawali/Computer-Vision-Project-Overview/blob/main/Computer%20Vision%20YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q ultralytics opencv-python
import ultralytics
ultralytics.checks()


Ultralytics 8.3.218 🚀 Python-3.13.7 torch-2.9.0 CPU (Apple M4)
Setup complete ✅ (10 CPUs, 16.0 GB RAM, 79.7/460.4 GB disk)


In [9]:
import os

os.environ["ROBOFLOW_API_KEY"] = input("Enter your Roboflow API key (hidden): ")



Enter your Roboflow API key (hidden): 4NCLkwV8v1vJiR82S5q4


In [15]:
!pip3 -q install roboflow
from roboflow import Roboflow
import os

rf = Roboflow(api_key=os.environ["ROBOFLOW_API_KEY"])

project  = rf.workspace("sawali").project("taco-dataset-ql1ng-by2kn")
version  = project.version(1)
dataset  = version.download("yolov8")

data_yaml = dataset.location + "/data.yaml"
print("YAML:", data_yaml)


loading Roboflow workspace...
loading Roboflow project...
YAML: /Users/darianaxelsawali/TACO-dataset-1/data.yaml


In [18]:
from pathlib import Path
import yaml

data_yaml = Path.home() / "Downloads" / "TACO dataset.v1i.yolov8" / "data.yaml"
print("YAML exists:", data_yaml.exists(), "\nPath:", data_yaml)

with open(data_yaml, "r") as f:
    y = yaml.safe_load(f)

print("nc:", y.get("nc"))
print("names:", y.get("names"))

YAML exists: True 
Path: /Users/darianaxelsawali/Downloads/TACO dataset.v1i.yolov8/data.yaml
nc: 6
names: ['cardboard', 'glass', 'metal', 'other', 'paper', 'plastic']


In [19]:
from ultralytics import YOLO
import torch

device = "mps" if torch.backends.mps.is_available() else "cpu"
print("Using device:", device)

Using device: mps


In [ ]:
model = YOLO("yolov8n.pt")  # nano = fastest

results = model.train(
    data=str(data_yaml),
    epochs=5,
    imgsz=512,
    batch=8,
    workers=0,
    patience=5,
    device=device
)

In [ ]:
metrics = model.val(device=device)   # uses val split from YAML
print(metrics.results_dict)

# Optional: evaluate test split if present in your dataset
test_metrics = model.val(split="test", device=device)
print("Test mAP50:", test_metrics.results_dict.get("metrics/mAP50(B)"))
print("Figures saved in:", test_metrics.save_dir)

In [ ]:
from glob import glob
demo_imgs = glob(str((data_yaml.parent / "test" / "images") / "*"))[:8]
preds = model.predict(source=demo_imgs, conf=0.25, save=True, device=device)
print("Annotated predictions saved in:", preds[0].save_dir)